In [76]:
import pandas
import random
import os
import json
import numpy as np
from datetime import datetime

# How much data do we want to make?
ROWCOUNT = 1000

## Part 1

### Customer

In [77]:
custFileName = "nameList1000.txt"

ROOT = os.path.dirname(os.getcwd()) + '/resource/' + custFileName

first_names = []
last_names = []

with open(ROOT, 'r') as file:
    rawnames = file.read().split('\n')

for name in rawnames:
    first_names.append(name[:name.find(' ')])
    last_names.append(name[1+name.find(' '):])

In [78]:
'''
--
-- given list of 1st and 2nd names
-- randomize them and duplicate them randomly for repeat customers
--
'''

# Create a list to hold all names
names = []

# Generate unique names (50% of 4000 = 2000 unique names)
for index in range(2000):
    first = random.choice(first_names)
    last = random.choice(last_names)
    name = f"{first} {last}"
    names.append(name)

# Generate repeated names (remaining 50% will be repeats 5-10 times)
for index in range(ROWCOUNT):  # Need 1000 more distinct names, each repeating 5-10 times
    first = random.choice(first_names)
    last = random.choice(last_names)
    name = f"{first} {last}"
    repeat_count = random.randint(5, 10)  # Repeat each name 5 to 10 times
    names.extend([name] * repeat_count)

# Shuffle the final list to mix unique and repeated names
random.shuffle(names)

# Ensure the list is exactly 1000 names
names = names[:ROWCOUNT]

# Convert the list to a single line with commas separating each name
# result = ", ".join(names)

In [79]:
'''
--
-- DF to pandas
--
'''
customerDict = {}
first_names = []
last_names = []

for name in names:
    first_names.append(name[:name.find(' ')])
    last_names.append(name[1+name.find(' '):])

customerDict["First"] = first_names
customerDict["Last"] = last_names

customerDF = pandas.DataFrame.from_dict(data=customerDict)
customerDF


,First,Last
0,Anna,Patel
1,Adeline,Foster
2,Nevaeh,Taylor
3,Nevaeh,Turner
4,Sarah,Howard
...,...,...
995,Scarlett,Jones
996,Genesis,Cox
997,Isla,Flores
998,Sarah,Perez


### Store

In [80]:
generateData = "genData.json"

ROOT = os.path.dirname(os.getcwd()) + '/resource/' + generateData

storeDict = {}

with open(ROOT, 'r') as file:
    data = json.load(file)
    data = data["Store"]

listAdd = []
for dict in data:
    listAdd.append(data[dict])

    
storeDf = pandas.DataFrame.from_records(listAdd)
storeDf

,Name,Address,City,Texas,Zip Code
0,South Austin Drive,102 South Austin Drive,Austin,Texas,72034
1,Western Drive,111 Western Drive,Houston,Texas,99345
2,North Lake,880 North Lake Drive,Dallas,Texas,14783


### Employee

In [81]:
generateData = "genData.json"

ROOT = os.path.dirname(os.getcwd()) + '/resource/' + generateData

with open(ROOT, 'r') as file:
    data = json.load(file)
    data = data["Employee"]

# get each stores employees
stores = []
for store in data:
    stores.append(data[store])


In [82]:
'''
{'one': {'EmployeeID': '1', 'ManagerID': 'Null', 'FirstName': 'Tom', 'LastName': 'Bates', 'StoreID': '1'}, 'two': {
'''


df = pandas.DataFrame(stores[0]).T
stores.pop(0)

for data in stores:
    dfNew = pandas.DataFrame(data).T
    df = pandas.concat([df,dfNew])

Employeedf = df
Employeedf.reset_index(drop=True, inplace=True)


### Product

In [83]:
generateData = "genData.json"

ROOT = os.path.dirname(os.getcwd()) + '/resource/' + generateData

with open(ROOT, 'r') as file:
    data = json.load(file)
    data = data["Product"]

# get each stores employees
product = []
for products in data:
    product.append(data[products])

productDF = pandas.DataFrame.from_records(product).T

productDF = productDF.rename(columns={0: 'Product', 1: 'Price'})

productDF['productID'] = productDF.reset_index().index + 1


### Full Table DF - Store

In [84]:
# Generate random integers between 0 and 10 for each row
random_counts = np.random.randint(0, 21, size=len(storeDf))
# random_counts

while storeDf.shape[0] < ROWCOUNT:
    
    # for each row number, count # of rows to add to df
    for row, count in enumerate(random_counts):
        
        # checker to prevent concatenating an empty list,
        if count > 0:
        
            # storeDf.concat([storeDf.loc[[i]] * count])
            row_to_duplicate = storeDf.loc[[row]]

            # Duplicate the row 2 times
            duplicated_row = pandas.concat([row_to_duplicate] * count, ignore_index=True)

            # Create a new DataFrame with the duplicated rows
            storeDf = pandas.concat([storeDf, duplicated_row], ignore_index=True)

# limit to 1000 rows
storeDf = storeDf.iloc[:ROWCOUNT]

### Full Table DF - Employee

In [85]:
# Generate random integers between 0 and 10 for each row
random_counts = np.random.randint(0, 21, size=len(Employeedf))
random_counts


array([ 0,  8, 20, 15, 19,  5, 12, 19,  1, 10, 15, 16,  9, 16])

In [86]:

while Employeedf.shape[0] < ROWCOUNT:
    
    # for each row number, count # of rows to add to df
    for row, count in enumerate(random_counts):
        # storeDf.concat([storeDf.loc[[i]] * count])
        
        # checker to prevent concatenating an empty list,
        if count > 0:
                
            row_to_duplicate = Employeedf.loc[[row]]

            # Duplicate the row 2 times
            duplicated_row = pandas.concat([row_to_duplicate] * count, ignore_index=True)

            # Create a new DataFrame with the duplicated rows
            Employeedf = pandas.concat([Employeedf, duplicated_row], ignore_index=True)

# limit to 1000 rows
Employeedf = Employeedf.iloc[:ROWCOUNT]

# Full table Df - Product

In [87]:
# Generate random integers between 0 and 10 for each row
random_counts = np.random.randint(0, 21, size=len(productDF))
random_counts


while productDF.shape[0] < ROWCOUNT:
    
    # for each row number, count # of rows to add to df
    for row, count in enumerate(random_counts):
        
        # checker to prevent concatenating an empty list,
        if count > 0:
                
            # storeDf.concat([storeDf.loc[[i]] * count])
            row_to_duplicate = productDF.loc[[row]]
                
            # Duplicate the row
            duplicated_row = pandas.concat([row_to_duplicate] * count, ignore_index=True)
            
            # Create a new DataFrame with the duplicated rows
            productDF = pandas.concat([productDF, duplicated_row], ignore_index=True)

# limit to 1000 rows
productDF = productDF.iloc[:ROWCOUNT]

## Concat ALl Tables into 1 table

In [88]:
fullTableDF = pandas.concat([customerDF,storeDf], axis=1)
fullTableDF = pandas.concat([fullTableDF,Employeedf], axis=1)
fullTableDF = pandas.concat([fullTableDF,productDF], axis=1)

In [107]:
# fullTableDF

## GCP Push